In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, mapping
from libpysal.weights import KNN
from spreg import ML_Lag
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import folium
import seaborn as sns
from libpysal.weights import KNN
from esda import Moran

In [3]:
property = gpd.read_file('data/cycle/Market_value_usable_hedonic_data.csv', engine='pyogrio')
canopy = gpd.read_file('data/canopy.gdb', engine='pyogrio')
ta = gpd.read_file('data/chc-boundaries/territorial-authority-2021-generalised.gpkg', engine='pyogrio')
sa2 = gpd.read_file('data/chc-boundaries/sa2/statistical-area-2-2023-generalised.shp')

/Users/rujalshrestha/Projects/chc-property-prices/venv/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [4]:
gdf = gpd.GeoDataFrame(
  property,
  geometry=gpd.points_from_xy(property['gd2000co'], property['gd2000_yco']),
  crs=4326
).to_crs(2193)

In [5]:
gdf = gdf.loc[:, ['YearSold', 'GrossSalePrice', 'geometry']]

In [6]:
pp = gdf[gdf['YearSold'].isin(['2017', '2018', '2019'])].drop(columns=['YearSold'])

In [7]:
pp.head()

,GrossSalePrice,geometry
3,550000,POINT (1564765.959 5177559.301)
7,480000,POINT (1563462.485 5177886.554)
25,555000,POINT (1563147.894 5178183.035)
35,500000,POINT (1563734.616 5176441.03)
38,635000,POINT (1563415.952 5177784.26)


In [8]:
pp['GrossSalePrice'] = pd.to_numeric(pp['GrossSalePrice'], errors='coerce')

In [25]:
pp['log_price'] = np.log(pp['GrossSalePrice'])

In [26]:
pp.head()

,GrossSalePrice,geometry,log_price
3,550000,POINT (1564765.959 5177559.301),13.217674
7,480000,POINT (1563462.485 5177886.554),13.081541
25,555000,POINT (1563147.894 5178183.035),13.226723
35,500000,POINT (1563734.616 5176441.03),13.122363
38,635000,POINT (1563415.952 5177784.26),13.361380


In [30]:
w = KNN.from_dataframe(pp, k=8)
w.transform = 'R'

In [32]:
mi = Moran(pp['log_price'], w)

print("Moran's I: ", mi.I)
print("Expected I: ", mi.EI)
print("p-value: ", mi.p_sim)

Moran's I:  0.577639515659582
Expected I:  -5.6189245378434565e-05
p-value:  0.001
